#### Planteamiento del Problema

Se desea predecir la cantidad de puntos que pueda hacer un jugador en un partido, 
a partir de estadísticas personales (del jugador) y colectivas (del equipo en donde juega). 

Imports

In [61]:
#Se importan todas las dependencias necesarias
import pandas as pd
import sqlalchemy
import psycopg2
import sqlite3
import csv
import os
import numpy as np
import scipy.stats

from statistics import mode
from scipy.stats import skew, kurtosis
from sqlalchemy import create_engine
from dotenv import load_dotenv
from nba_api.stats.endpoints import playergamelogs, teamgamelogs, teamgamelog, playerdashboardbylastngames

Obtenemos las estadísticas de los jugadores por partidos en las temporadas 2020-21, 2021-22 y 2022-23

In [2]:
season0_players = playergamelogs.PlayerGameLogs(season_nullable='2020-21') #Consulta para temporada 2020-21
season1_players = playergamelogs.PlayerGameLogs(season_nullable='2021-22') #Consulta para temporada 2021-22
season2_players = playergamelogs.PlayerGameLogs(season_nullable='2022-23') #Consulta para temporada 2022-23
players = pd.concat([season0_players.get_data_frames()[0],season1_players.get_data_frames()[0],season2_players.get_data_frames()[0]], ignore_index=True) #Dataframe de estadísticas de jugadores por partido en las tres temporadas
players.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG
0,2020-21,1629611,Terance Mann,Terance,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,...,1,4919,3793,3538,1834,9325,1849,143,8568,1
1,2020-21,1629725,Jeremiah Martin,Jeremiah,1610612739,CLE,Cleveland Cavaliers,0022001067,2021-05-16T00:00:00,CLE @ BKN,...,15321,1,16020,16353,4365,15681,1849,143,16754,1
2,2020-21,1629684,Grant Williams,Grant,1610612738,BOS,Boston Celtics,0022001073,2021-05-16T00:00:00,BOS @ NYK,...,1,4919,6289,7127,21655,10324,1849,143,8568,1
3,2020-21,1626220,Royce O'Neale,Royce,1610612762,UTA,Utah Jazz,0022001077,2021-05-16T00:00:00,UTA @ SAC,...,1,10833,10308,10740,4365,13985,1849,143,12215,1
4,2020-21,1629002,Chimezie Metu,Chimezie,1610612758,SAC,Sacramento Kings,0022001077,2021-05-16T00:00:00,SAC vs. UTA,...,15321,10833,6289,11759,19248,12582,1849,143,12891,1


Obtenemos las estadísticas de los equipos por partidos en las temporadas 2020-21, 2021-22 y 2022-23

In [114]:
season0_teams = teamgamelogs.TeamGameLogs(season_nullable='2020-21') #Consulta para temporada 2020-21
season1_teams = teamgamelogs.TeamGameLogs(season_nullable='2021-22') #Consulta para temporada 2021-22
season2_teams = teamgamelogs.TeamGameLogs(season_nullable='2022-23') #Consulta para temporada 2022-23
teams = pd.concat([season0_teams.get_data_frames()[0],season1_teams.get_data_frames()[0],season2_teams.get_data_frames()[0]], ignore_index=True) #Dataframe de estadísticas de equipos por partido en las tres temporadas
teams.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG
0,2020-21,1610612757,POR,Portland Trail Blazers,0022001076,2021-05-16T00:00:00,POR vs. DEN,W,48.0,46,...,1099,1048,2016,490,141,376,786,115,280,1
1,2020-21,1610612753,ORL,Orlando Magic,0022001075,2021-05-16T00:00:00,ORL @ PHI,L,48.0,36,...,1276,1636,1867,1499,1024,1927,61,714,1644,1
2,2020-21,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,L,48.0,39,...,1619,641,751,1499,332,49,1002,960,1693,1
3,2020-21,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16T00:00:00,ATL vs. HOU,W,48.0,49,...,146,826,348,155,1024,376,1220,354,60,1
4,2020-21,1610612763,MEM,Memphis Grizzlies,0022001070,2021-05-16T00:00:00,MEM @ GSW,L,48.0,40,...,938,826,126,801,663,1376,1002,1723,1693,1


Agregamos la columna PTSR (Puntos recibidos) por el equipo en cada partido, temporadas 2020-21, 2021-22 y 2022-23. 

Sería restar los puntos realizados menos la diferencia final de puntaje respecto al otro equipo una vez finalizado el partido.

In [115]:
teams['PTSR'] = (teams['PTS'] - teams['PLUS_MINUS']).astype(int)
teams.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG,PTSR
0,2020-21,1610612757,POR,Portland Trail Blazers,0022001076,2021-05-16T00:00:00,POR vs. DEN,W,48.0,46,...,1048,2016,490,141,376,786,115,280,1,116
1,2020-21,1610612753,ORL,Orlando Magic,0022001075,2021-05-16T00:00:00,ORL @ PHI,L,48.0,36,...,1636,1867,1499,1024,1927,61,714,1644,1,128
2,2020-21,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,L,48.0,39,...,641,751,1499,332,49,1002,960,1693,1,125
3,2020-21,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16T00:00:00,ATL vs. HOU,W,48.0,49,...,826,348,155,1024,376,1220,354,60,1,95
4,2020-21,1610612763,MEM,Memphis Grizzlies,0022001070,2021-05-16T00:00:00,MEM @ GSW,L,48.0,40,...,826,126,801,663,1376,1002,1723,1693,1,113


Hacemos una nueva consulta a la API con otro endpoint para poder obtener el W_PCT o % de Victorias del equipo

In [116]:
teams_list = teams.TEAM_ID.unique().tolist() #El for será para encontrar el W_PCT de todos los equipos para cada partido de la temporada
teamLog = pd.DataFrame() #inicializamos para que no quede basura

for team in teams_list:
    teamLog0 = teamgamelog.TeamGameLog(season='2020-21',team_id=team) #Consulta para temporada 2020-21
    teamLog1 = teamgamelog.TeamGameLog(season='2021-22',team_id=team) #Consulta para temporada 2021-22
    teamLog2 = teamgamelog.TeamGameLog(season='2022-23',team_id=team) #Consulta para temporada 2022-23
    teamLog3 = pd.concat([teamLog0.get_data_frames()[0], teamLog1.get_data_frames()[0], teamLog2.get_data_frames()[0]],ignore_index=True) #En cada ciclo se forma el dataframe por equipo por partido en las tres temporadas 
    teamLog  = pd.concat([teamLog, teamLog3]) #Se van consolidando en un dataframe las consultas para todos los equipos

In [117]:
teamLog.head()

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1610612757,0022001076,"MAY 16, 2021",POR vs. DEN,W,42,30,0.583,240,46,...,0.846,11,40,51,24,3,6,13,16,132
1,1610612757,0022001053,"MAY 13, 2021",POR @ PHX,L,41,30,0.577,240,44,...,0.778,1,32,33,23,7,3,13,18,117
2,1610612757,0022001045,"MAY 12, 2021",POR @ UTA,W,41,29,0.586,240,40,...,0.706,9,30,39,17,7,6,5,18,105
3,1610612757,0022001031,"MAY 10, 2021",POR vs. HOU,W,40,29,0.580,240,47,...,0.800,12,35,47,30,7,4,9,18,140
4,1610612757,0022001018,"MAY 08, 2021",POR vs. SAS,W,39,29,0.574,240,45,...,0.733,18,45,63,24,3,5,8,15,124


In [118]:
#Se renombran las columnas de con el mismo titulo que en el dataframe de teams, para poder hacer el merge deseado (filtrando por las columnas GAME_ID y TEAM_ID)
teamLog = teamLog.rename(columns={'Game_ID': 'GAME_ID'})
teamLog = teamLog.rename(columns={'Team_ID': 'TEAM_ID'})

In [119]:
#Se hace el merge deseado de W_PCT (% Victorias) filtrando por las columnas GAME_ID y TEAM_ID
merged_df = pd.merge(teams,teamLog, on=['GAME_ID','TEAM_ID'], how='left')
teams['W_PCT'] = merged_df['W_PCT']
teams.head()

,SEASON_YEAR,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG,PTSR,W_PCT
0,2020-21,1610612757,POR,Portland Trail Blazers,0022001076,2021-05-16T00:00:00,POR vs. DEN,W,48.0,46,...,2016,490,141,376,786,115,280,1,116,0.583
1,2020-21,1610612753,ORL,Orlando Magic,0022001075,2021-05-16T00:00:00,ORL @ PHI,L,48.0,36,...,1867,1499,1024,1927,61,714,1644,1,128,0.292
2,2020-21,1610612761,TOR,Toronto Raptors,0022001079,2021-05-16T00:00:00,TOR vs. IND,L,48.0,39,...,751,1499,332,49,1002,960,1693,1,125,0.375
3,2020-21,1610612737,ATL,Atlanta Hawks,0022001066,2021-05-16T00:00:00,ATL vs. HOU,W,48.0,49,...,348,155,1024,376,1220,354,60,1,95,0.569
4,2020-21,1610612763,MEM,Memphis Grizzlies,0022001070,2021-05-16T00:00:00,MEM @ GSW,L,48.0,40,...,126,801,663,1376,1002,1723,1693,1,113,0.528


In [120]:
#Para comprobar que no hayan quedado valores que no hayan hecho math entre columna GAME_ID y TEAM_ID
#Si fuese el caso, se hubiese rellenado con NaN
teams.W_PCT.isna().any()

False

Obtenemos un solo DataFrame con las columnas que necesitamos, se va a consolidar todo en el Dataframe de estadísticas de los jugadores "players"

In [121]:
#Del Dataframe "teams" se obtienen las columnas de Puntos recibidos por el equipo 'PTSR', 'PTS' Puntos realizados por el equipo y W_PCT (%Victorias)
#Se hace el merge deseado de Puntos recibidos por el equipo 'PTSR', 'PTS' Puntos realizados por el equipo y W_PCT (%Victorias); filtrando por las columnas GAME_ID y TEAM_ID
teams = teams.rename(columns={'PTS': 'PTSTeam'}) #Porque en el Dataframe players tambien hay una columna PTS

merged_df2 = pd.merge(players,teams, on=['GAME_ID','TEAM_ID'], how='left')

players[['PTSRTeam','PTSTeam','W_PCTTeam']] = merged_df2[['PTSR','PTSTeam','W_PCT']] #Se crean las nuevas columnas en el Dataframe players de acuerdo al merge realizado.

players.head()

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PTS_RANK,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,PTSRTeam,PTSTeam,W_PCTTeam
0,2020-21,1629611,Terance Mann,Terance,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,...,3538,1834,9325,1849,143,8568,1,117,112,0.653
1,2020-21,1629725,Jeremiah Martin,Jeremiah,1610612739,CLE,Cleveland Cavaliers,0022001067,2021-05-16T00:00:00,CLE @ BKN,...,16353,4365,15681,1849,143,16754,1,123,109,0.306
2,2020-21,1629684,Grant Williams,Grant,1610612738,BOS,Boston Celtics,0022001073,2021-05-16T00:00:00,BOS @ NYK,...,7127,21655,10324,1849,143,8568,1,96,92,0.500
3,2020-21,1626220,Royce O'Neale,Royce,1610612762,UTA,Utah Jazz,0022001077,2021-05-16T00:00:00,UTA @ SAC,...,10740,4365,13985,1849,143,12215,1,99,121,0.722
4,2020-21,1629002,Chimezie Metu,Chimezie,1610612758,SAC,Sacramento Kings,0022001077,2021-05-16T00:00:00,SAC vs. UTA,...,11759,19248,12582,1849,143,12891,1,121,99,0.431


In [122]:
#Para comprobar que no hayan quedado valores que no hayan hecho math entre columna GAME_ID y TEAM_ID
#Si fuese el caso, se hubiese rellenado con NaN
print(players.PTSRTeam.isna().any())
print(players.PTSTeam.isna().any())
print(players.W_PCTTeam.isna().any())

False
False
False


Se agrega una nueva columna para determinar si un equipo es local o visitante

In [123]:
def determinar_tipo(data):
    if '@' in data:
        return 'A'  # Si hay '@', entonces es visitante
    elif 'vs' in data:
        return 'H'  # Si hay 'vs', entonces es local
    
players['HOME_AWAY'] = players['MATCHUP'].apply(determinar_tipo) #Nueva columna Home (H), Away (A)

players.head(2)

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,PLUS_MINUS_RANK,NBA_FANTASY_PTS_RANK,DD2_RANK,TD3_RANK,WNBA_FANTASY_PTS_RANK,AVAILABLE_FLAG,PTSRTeam,PTSTeam,W_PCTTeam,HOME_AWAY
0,2020-21,1629611,Terance Mann,Terance,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,...,1834,9325,1849,143,8568,1,117,112,0.653,A
1,2020-21,1629725,Jeremiah Martin,Jeremiah,1610612739,CLE,Cleveland Cavaliers,0022001067,2021-05-16T00:00:00,CLE @ BKN,...,4365,15681,1849,143,16754,1,123,109,0.306,A


Cargamos los datos en un .CSV

In [ ]:
players.to_csv('/workspace/ProyectoDS/data/raw/data_raw.csv', index=False)

Removemos las columnas que no se utilizarán en la generación del modelo predictivo.

In [124]:
players.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK', 'AVAILABLE_FLAG', 'PTSRTeam', 'PTSTeam',
       'W_PCTTeam', 'HOME_AWAY'],
      dtype='object')

In [125]:
#Remocion de columnas
players = players.drop(['WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK',
       'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK',
       'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK',
       'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK',
       'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK',
       'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK',
       'WNBA_FANTASY_PTS_RANK','NBA_FANTASY_PTS', 'DD2',
       'TD3'], axis = 1)

players.head(2)

,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,...,BLKA,PF,PFD,PTS,PLUS_MINUS,AVAILABLE_FLAG,PTSRTeam,PTSTeam,W_PCTTeam,HOME_AWAY
0,2020-21,1629611,Terance Mann,Terance,1610612746,LAC,LA Clippers,0022001074,2021-05-16T00:00:00,LAC @ OKC,...,0,1,3,19,16,1,117,112,0.653,A
1,2020-21,1629725,Jeremiah Martin,Jeremiah,1610612739,CLE,Cleveland Cavaliers,0022001067,2021-05-16T00:00:00,CLE @ BKN,...,1,0,0,4,9,1,123,109,0.306,A


Tomamos el promedio de los últimos 5 partidos para las estadísticas por jugador

In [126]:
#Ordenar el Dataframe de forma ascendente en base a las columnas descritas.
players = players.sort_values(by=['SEASON_YEAR','PLAYER_ID','GAME_DATE'], ascending=[True,True,True])

In [127]:
#Hacemos reset el indice del dataframe en caso de que lo podamos necesitar en orden
players = players.reset_index(drop=True)

In [128]:
players.shape

(74987, 38)

In [ ]:
def calcular_promedio_puntos(dataframe, columna, player_id, game_date, n_partidos):
    """
    Calcula el promedio de puntos de un jugador en los N partidos anteriores, excluyendo el partido actual.
    Parámetros:
      dataframe: El dataframe con los datos de los jugadores.
      game_date: El GAME_ID del partido actual.
      player_id: El PLAYER_ID del jugador para el que se quiere calcular el promedio.
      n_partidos: El número de partidos anteriores a considerar.
      columna: La columna que contiene los puntos del jugador.
    Devuelve:
      El promedio de puntos del jugador en los N partidos anteriores.
    """
    # Filtrar el dataframe para los partidos anteriores al actual y del jugador específico
    partidos_anteriores_jugador = dataframe[(dataframe['GAME_DATE'] < game_date) & (dataframe['PLAYER_ID'] == player_id)]
    
    # Si no hay suficientes partidos anteriores para el jugador, tomar los partidos disponibles
    n_partidos_disponibles = min(len(partidos_anteriores_jugador), n_partidos)
    
    # Si hay al menos un partido anterior, calcular el promedio de puntos
    if n_partidos_disponibles > 0:
        promedio_puntos = partidos_anteriores_jugador.tail(n_partidos_disponibles)[columna].mean()
    else:
        # Si no hay partidos anteriores disponibles, tomar el valor del partido actual
        promedio_puntos = dataframe[(dataframe['GAME_DATE'] == game_date) & (dataframe['PLAYER_ID'] == player_id)][columna].iloc[0]
    
    return promedio_puntos

#Lista de columnas para las cuales se obtendrán las estadísticas promediadas
lista_columnas_para_promedio = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'PTSRTeam',
       'PTSTeam', 'W_PCTTeam']

#Tomamos una listas de los jugadores y de las fechas de los partidos.
jugadores_ID = players.PLAYER_ID.unique().tolist()
dates = players.GAME_DATE.unique().tolist()


n_partidos = 5 #numero de partidos para sacar el promedio

#Se llama a la funcion para obtener las estadisticas promediadas de todos los jugadores
for columna in lista_columnas_para_promedio:
  for jugador in jugadores_ID:
      for date in dates:
        
        indice = players.index[(players['GAME_DATE']==date) & (players['PLAYER_ID']==jugador)]

        if indice.empty:
                # Si no se encontró un índice válido, pasar a la siguiente fecha
          continue

        promedio = calcular_promedio_puntos(players, columna, jugador, date, n_partidos)

        if promedio is None:
                # Pasar a la siguiente fecha
          continue
              
        players.loc[indice,columna+'_'+'PROM'] = promedio

In [ ]:
players.to_csv('/workspace/ProyectoDS/data/raw/data_complete_raw.csv', index=False)

Cargo el nuevo CSV completo.

In [ ]:
players = pd.read_csv('/workspace/ProyectoDS/data/raw/data_complete_raw.csv')
players.head()

Enviamos los registros a una DB.

In [1]:

players = pd.read_csv('/workspace/ProyectoDS/data/raw/data_complete_raw.csv')
players_records = players.to_records(index=False)


# A database named "NBA.db" is generated in the current directory
con = sqlite3.connect("NBA.db")
con.execute("DROP TABLE IF EXISTS ESTADISTICAS;")

players.to_sql('ESTADISTICAS',con, index=True,if_exists='replace')

con.close()

In [20]:
con = sqlite3.connect("NBA.db")
cur = con.cursor()

# -- Obtenemos los 10 máximos anotadores.
cur.execute("""SELECT PLAYER_NAME, TEAM_NAME,
            PTS, FGM, FGA, FG_PCT,FG3M, FG3A, FG3_PCT
            FROM ESTADISTICAS
            ORDER BY PTS DESC
            LIMIT 10
            ;

            """)

res = cur.fetchall()

for row in res:
    print(f"Nombre {row[0]}")
    print(f"Equipo: {row[1]}")
    print(f"Puntos: {row[2]}")
    print(f"Tiros convertidos: {row[3]}/{row[4]}")
    print(f"Porcentaje de aciertos: {row[5]}")
    print(f"Tiros de 3: {row[6]}/{row[7]}")
    print(f"Porcentaje tiros de 3: {row[8]}\n\n")


Nombre Damian Lillard
Equipo: Portland Trail Blazers
Puntos: 71
Tiros convertidos: 22/38
Porcentaje de aciertos: 0.579
Tiros de 3: 13/22
Porcentaje tiros de 3: 0.591


Nombre Donovan Mitchell
Equipo: Cleveland Cavaliers
Puntos: 71
Tiros convertidos: 22/34
Porcentaje de aciertos: 0.647
Tiros de 3: 7/15
Porcentaje tiros de 3: 0.467


Nombre Stephen Curry
Equipo: Golden State Warriors
Puntos: 62
Tiros convertidos: 18/31
Porcentaje de aciertos: 0.581
Tiros de 3: 8/16
Porcentaje tiros de 3: 0.5


Nombre Bradley Beal
Equipo: Washington Wizards
Puntos: 60
Tiros convertidos: 20/35
Porcentaje de aciertos: 0.571
Tiros de 3: 7/10
Porcentaje tiros de 3: 0.7


Nombre Jayson Tatum
Equipo: Boston Celtics
Puntos: 60
Tiros convertidos: 20/37
Porcentaje de aciertos: 0.541
Tiros de 3: 5/7
Porcentaje tiros de 3: 0.714


Nombre Kyrie Irving
Equipo: Brooklyn Nets
Puntos: 60
Tiros convertidos: 20/31
Porcentaje de aciertos: 0.645
Tiros de 3: 8/12
Porcentaje tiros de 3: 0.667


Nombre Karl-Anthony Towns
Equipo

In [25]:
    # -- Obtenemos los puntos de jugador por temporada.
cur.execute("""SELECT PLAYER_NAME, TEAM_NAME, SEASON_YEAR,
              SUM(PTS) AS PTS_TOT
              FROM ESTADISTICAS
              GROUP BY PLAYER_NAME, TEAM_NAME, SEASON_YEAR
              ORDER BY PTS_TOT DESC
              LIMIT 10""")

res = cur.fetchall()

for row in res:
        print(f"Nombre {row[0]}")
        print(f"Equipo: {row[1]}")
        print(f"Temporada: {row[2]}")
        print(f"Puntos totales: {row[3]}\n\n")

con.close()

Nombre Jayson Tatum
Equipo: Boston Celtics
Temporada: 2022-23
Puntos totales: 2225


Nombre Joel Embiid
Equipo: Philadelphia 76ers
Temporada: 2022-23
Puntos totales: 2183


Nombre Trae Young
Equipo: Atlanta Hawks
Temporada: 2021-22
Puntos totales: 2155


Nombre Luka Doncic
Equipo: Dallas Mavericks
Temporada: 2022-23
Puntos totales: 2138


Nombre Shai Gilgeous-Alexander
Equipo: Oklahoma City Thunder
Temporada: 2022-23
Puntos totales: 2135


Nombre DeMar DeRozan
Equipo: Chicago Bulls
Temporada: 2021-22
Puntos totales: 2118


Nombre Joel Embiid
Equipo: Philadelphia 76ers
Temporada: 2021-22
Puntos totales: 2079


Nombre Jayson Tatum
Equipo: Boston Celtics
Temporada: 2021-22
Puntos totales: 2046


Nombre Stephen Curry
Equipo: Golden State Warriors
Temporada: 2020-21
Puntos totales: 2015


Nombre Nikola Jokic
Equipo: Denver Nuggets
Temporada: 2021-22
Puntos totales: 2004




Analizamos variables numéricas

In [43]:
numeric_columns = []
for column_name in players:
    if players[column_name].dtype == 'int64' or players[column_name].dtype == 'float64':
        numeric_columns.append(column_name)


['TEAM_ID', 'GAME_ID', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'AVAILABLE_FLAG', 'PTSRTeam', 'PTSTeam', 'W_PCTTeam', 'MIN_PROM', 'FGM_PROM', 'FGA_PROM', 'FG_PCT_PROM', 'FG3M_PROM', 'FG3A_PROM', 'FG3_PCT_PROM', 'FTM_PROM', 'FTA_PROM', 'FT_PCT_PROM', 'OREB_PROM', 'DREB_PROM', 'REB_PROM', 'AST_PROM', 'TOV_PROM', 'STL_PROM', 'BLK_PROM', 'BLKA_PROM', 'PF_PROM', 'PFD_PROM', 'PTS_PROM', 'PLUS_MINUS_PROM', 'PTSRTeam_PROM', 'PTSTeam_PROM', 'W_PCTTeam_PROM']


Muestro la media, mínimo, máximo, rango, desviación estándar, cuartiles, curtosis y moda.

In [73]:
players['FGM'].mode()

0    1
Name: FGM, dtype: int64

In [77]:
for column_name in numeric_columns:
    mean = players[column_name].mean()
    min = players[column_name].min()
    max = players[column_name].max()
    moda = players[column_name].mode()
    rang = max - min 
    std = players[column_name].std()
    skewness = skew(players[column_name])
    cuartil_25_normal = np.percentile(players[column_name],25)
    cuartil_50_normal = np.percentile(players[column_name],50)
    cuartil_75_normal = np.percentile(players[column_name],75)
    kurtosis = scipy.stats.kurtosis(players[column_name])

    print(f'Columna: {column_name}')
    print(f'    Media: {mean}')
    print(f'    Moda: {moda[0]}')
    print(f'    Mínimo: {min}')
    print(f'    Máximo: {max}')
    print(f'    Rango: {rang}')
    print(f'    Desviación estándar: {std}')
    print(f'    Cuatril 25: {cuartil_25_normal}')
    print(f'    Cuatril 25: {cuartil_50_normal}')
    print(f'    Cuatril 25: {cuartil_75_normal}')
    print(f'    Kurtosis: {kurtosis}')
    print(f'    Asimetría: {skewness}')


Columna: TEAM_ID
    Media: 1610612751.4898982
    Moda: 1610612743
    Mínimo: 1610612737
    Máximo: 1610612766
    Rango: 29
    Desviación estándar: 8.657299333755455
    Cuatril 25: 1610612744.0
    Cuatril 25: 1610612751.0
    Cuatril 25: 1610612759.0
    Kurtosis: -1.209389232138601
    Asimetría: -0.0010050956305153462
Columna: GAME_ID
    Media: 22104376.027271394
    Moda: 22000700
    Mínimo: 22000001
    Máximo: 22201230
    Rango: 201229
    Desviación estándar: 80734.13040925466
    Cuatril 25: 22000879.0
    Cuatril 25: 22100673.0
    Cuatril 25: 22200334.0
    Kurtosis: -1.4614406339625774
    Asimetría: -0.0692594083277923
Columna: MIN
    Media: 22.8052862496166
    Moda: 12.0
    Mínimo: 0.0
    Máximo: 56.516666666666666
    Rango: 56.516666666666666
    Desviación estándar: 10.706909688751281
    Cuatril 25: 15.183333333333334
    Cuatril 25: 23.738333333333333
    Cuatril 25: 31.51333333333333
    Kurtosis: -0.8057228032301671
    Asimetría: -0.27899357157324634
C

In [74]:
players.FTA.value_counts()

FTA
0     33122
2     15002
4      6547
1      4527
3      4353
6      2996
5      2693
7      1492
8      1424
9       813
10      673
11      403
12      302
13      187
14      151
15      115
16       61
17       38
18       30
19       21
20       12
21        9
22        7
24        3
23        2
25        2
27        1
28        1
Name: count, dtype: int64

Analizamos las variables categóricas

In [84]:
categ_columns = players.select_dtypes(include=['object','category']).columns

categ_columns = categ_columns.tolist()
categ_columns.append('AVAILABLE_FLAG')
categ_columns = pd.Index(categ_columns)
categ_columns

Index(['SEASON_YEAR', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ABBREVIATION',
       'TEAM_NAME', 'GAME_DATE', 'MATCHUP', 'WL', 'HOME_AWAY',
       'AVAILABLE_FLAG'],
      dtype='object')